#MNIST 예시

In [4]:
import torch
# 신경망 구축에 필요한 여러 매서드를 담은 torch.nn
import torch.nn as nn
# torch.nn의 모든 함수를 포함, 손실/활성화/풀링/합성곱/선형 및 기타 신경망 함수 가 포함됨
import torch.nn.functional as F
# 최적화(신경망의 가중치,매개변수 조정을 위해 오차를 역전파 하는 과정) 모듈이 담김
import torch.optim as optim

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
class ConvNet(nn.Module):
  """
  kernel_size : 보통 홀수. 너무 작으면 픽셀을 처리하는 kernel이 이웃 픽셀의 정보를 가지지 못한다.너무 크면 이미지 내에서 정밀하지
                않은 특징을 얻게 됨. 작은 kernel_size의 많은 layer를 쓰면 네트워크가 깊어지고, 더 복잡한 특징 학습 가능.

  feature map : 이미지 데이터에서 픽셀 정보를 담고 있는 channel(차원)을 의미. 이미지에서 더 많은 특징을 추출하려거든 channel을 크게
                하면 된다.

  input shape이 28x28x1임

  Conv2d(input_channel, output_channel, kernel_size, stride)


  """
  # 각 layer의 뉴런개수 및 layer들 정의
  def __init__(self):
    super(ConvNet, self).__init__()

    # 합성곱 layer
    self.cn1 = nn.Conv2d(1,16,3,1)
    self.cn2 = nn.Conv2d(16,32,3,1)
    # 드롭아웃 layer
    self.dp1 = nn.Dropout(0.1)
    self.dp2 = nn.Dropout(0.25)
    # fully-connected(fc) layer, 4608=12x12x32
    self.fc1 = nn.Linear(4608, 64)
    # 최종 출력은 10개 클래스 중 하나
    self.fc2 = nn.Linear(64,10)

  def forward(self, x):
    x = self.cn1(x)
    x = F.relu(x)
    x = self.cn2(x)
    x = F.relu(x)
    # kernel size가 2x2
    x = F.max_pool2d(x,2)
    x = self.dp1(x)
    # 1차원 백터로 평면화
    x = torch.flatten(x,1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dp2(x)
    x = self.fc2(x)
    # 모델의 예측을 out에 담음
    out = F.log_softmax(x, dim=1)
    return out